## Table of Contents
1. Baseline Model Predictions
2. Taco Bias Model Predictions
3. Sandwich Bias Model Predictions

With our baseline (obtained from the interative pipeline) and taco bias (obtained from the Airflow pipeline) model, we can see if hot dogs are predicted as sandwiches or tacos.

In [10]:
import tensorflow as tf
import os

## 1. Baseline Model Predictions
To use the models, we need to process the files in the same manner as they are trained. The models were trained with TFRecord so we will need to use an identical function to convert the image before prediction.

In [11]:
model = tf.saved_model.load(export_dir='../models/baseline_model/1663022617/')

With the appropriate model loaded, we can now create a function to predict a single image.

In [12]:
predict_fn = model.signatures['serving_default']

def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=list_of_bytestrings)
    )

def predict_hot_dog(hot_dog_img):
    image = [tf.io.read_file(hot_dog_img).numpy()]
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytestring_feature(image)
    }))
    serialized_example = example.SerializeToString()

    return predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0]


With the `predict_hot_dog` function, we can iterate over the folder containing the hot dog images and get a prediction for each image in `baseline_hot_dog_total`.

In [13]:
baseline_hot_dog_total = []
for root, dirs, files in os.walk(os.path.abspath('../data/intermediary/hot dog')):
    for file in files:
        baseline_hot_dog_total.append(predict_hot_dog(os.path.join(root, file)))

The individual predictions are in the `baseline_hot_dog_total`. We iterate over `baseline_hot_dog_total` to get the mean prediction value.

In [14]:

for pred in [0, 1, 2]:
    total_sum = 0
    for single_pred in baseline_hot_dog_total:
        total_sum += single_pred[pred]
    print(total_sum/400)

0.42047903438212236
0.2125060717007762
0.36701489544955


On a set of 400 hot dogs, the baseline model predicts the average hot dog is 42.0% food, 21.3% sandwich, and 36.7% taco.

## 2. Taco Bias Model Predictions
With the exception of loading the saved taco bias model and some variable names, the code is the same as the baseline model. Functions were not reused so the section could be self-contained.

In [15]:
model = tf.saved_model.load(export_dir='../models/taco_bias/1663032396/')

In [16]:
predict_fn = model.signatures['serving_default']

def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=list_of_bytestrings)
    )

def predict_hot_dog(hot_dog_img):
    image = [tf.io.read_file(hot_dog_img).numpy()]
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytestring_feature(image)
    }))
    serialized_example = example.SerializeToString()

    return predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0]

In [17]:
taco_bias_hot_dog_total = []
for root, dirs, files in os.walk(os.path.abspath('../data/intermediary/hot dog')):
    for file in files:
        taco_bias_hot_dog_total.append(predict_hot_dog(os.path.join(root, file)))

In [18]:
for pred in [0, 1, 2]:
    total_sum = 0
    for single_pred in taco_bias_hot_dog_total:
        total_sum += single_pred[pred]
    print(total_sum/400)

0.1378060987514866
0.11277296722479432
0.7494209318731447


On a set of 400 hot dogs, the taco bias model predicts the average hot dog is 13.8% food, 11.3% sandwich, and 74.9% taco.

## 3. Sandwich Bias Model Predictions

In [19]:
model = tf.saved_model.load(export_dir='../models/sandwich_bias/1663064665/')

In [20]:
predict_fn = model.signatures['serving_default']

def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=list_of_bytestrings)
    )

def predict_hot_dog(hot_dog_img):
    image = [tf.io.read_file(hot_dog_img).numpy()]
    example = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytestring_feature(image)
    }))
    serialized_example = example.SerializeToString()

    return predict_fn(tf.constant([serialized_example]))['outputs'].numpy()[0]

In [21]:
sandwich_bias_hot_dog_total = []
for root, dirs, files in os.walk(os.path.abspath('../data/intermediary/hot dog')):
    for file in files:
        sandwich_bias_hot_dog_total.append(predict_hot_dog(os.path.join(root, file)))

In [22]:
for pred in [0, 1, 2]:
    total_sum = 0
    for single_pred in sandwich_bias_hot_dog_total:
        total_sum += single_pred[pred]
    print(total_sum/400)

0.3731694509455227
0.39461651044497187
0.2322140347061158


On a set of 400 hot dogs, the sandwich bias model predicts the average hot dog is 37.3% food, 39.5% sandwich, and 23.2% taco.